# Randomised Iterative Improvement on TSP



Set classpath to the pre-compiled jar

In [ ]:
@file:DependsOn("../build/libs/kglsm.jar")
@file:DependsOn("ch.qos.logback:logback-classic:1.2.3")
@file:DependsOn("ch.qos.logback:logback-core:1.2.3")
@file:DependsOn("io.github.microutils:kotlin-logging-jvm:2.0.3")

Define imports

In [ ]:
import com.sihvi.glsm.problem.TSP
import com.sihvi.glsm.sls.GLSMBuilder
import com.sihvi.glsm.sls.StateMachineTransition
import com.sihvi.glsm.space.PermutationSearchSpace
import com.sihvi.glsm.space.SearchSpace
import com.sihvi.glsm.strategy.IIMode
import com.sihvi.glsm.strategy.IterativeImprovementStrategy
import com.sihvi.glsm.strategy.RandomWalkStrategy
import com.sihvi.glsm.transitionpredicate.FixedIterationPredicate
import com.sihvi.glsm.transitionpredicate.NotPredicate
import com.sihvi.glsm.transitionpredicate.ProbabilisticPredicate
import com.sihvi.glsm.memory.Memory
import com.sihvi.glsm.memory.BasicMemory
import com.sihvi.glsm.memory.BasicSolution
import com.sihvi.glsm.memory.attribute.Stash
import com.sihvi.glsm.memory.attribute.MemoryAttribute
import com.sihvi.glsm.problem.CostFunction

## Problem

We are using TSP problem as an example. Get an instance of the problem.

We use a280 instance from TSPLIB [

In [ ]:
val dimensions = 2

val problemInstance = TSP.fromFile("a280.tsp")
val noPoints = problemInstance.size
println(problemInstance)

## Search space
Permutation search space defines two methods on an array of booleans: to get a neighbourhood and a random neighbour.

2-exchange neighbourhood is used for permutation search space

In [ ]:
val space = PermutationSearchSpace(noPoints)

## Memory
Basic memory that holds:
* Current solution and its cost
* Best solution and best cost
* Number of steps performed
* Number of steps performed without improvement (when it was expected)

In [ ]:
val initialSolution = space.getInitial()
val attributes = listOf<MemoryAttribute>(Stash<BasicSolution<Int>>())
val memory = BasicMemory(BasicSolution(initialSolution, problemInstance.evaluate(initialSolution)), attributes)
memory

## GLSM and Strategies
Randomised Iterative Improvement consists of two strategies that are flipped probabilistically
* Iterative Best Improvement Strategy -- picks a solution from a neighbourhood that gives the best improvement
* Random Walk Strategy -- randomly picks a solution from a neighbourhood

The termination predicate of choice here is No Improvement Predicate, i.e. we terminate the search if there were n steps without improvement (in this case 10)

In [ ]:
val terminationPredicate = FixedIterationPredicate(500)

val wp = 0.01
val toRandomPredicate = ProbabilisticPredicate(to = wp)
val toIIPredicate = NotPredicate(toRandomPredicate)

val walk = RandomWalkStrategy<Int>()
val iterativeImprovement = IterativeImprovementStrategy<Int>(IIMode.FIRST, true)

As all the components are defined, we can now build the GLSM with strategies and transitions between them 

In [ ]:
@Suppress("UNUSED_PARAMETER")
fun <T> pushStash(memory: Memory<T, BasicSolution<T>>, searchSpace: SearchSpace<T>, costFunction: CostFunction<T>) {
    val stash: Stash<BasicSolution<T>> = memory.getAttribute()
    stash.addToStash(memory.bestSolution)
}

In [ ]:
val glsm = GLSMBuilder<Int, BasicSolution<Int>>()
        .addStrategy(iterativeImprovement)
        .addStrategy(walk)
        .addTransition(StateMachineTransition(0, 1, toRandomPredicate))
        .addTransition(StateMachineTransition(1, 0, toIIPredicate))
        .addTransition(StateMachineTransition(0, -1, terminationPredicate))
        .build()

In [ ]:
glsm.toASCII()

# Solve

With everything ready we can run GLSM on our problem instance

In [ ]:
val finalSolution = glsm.solve(memory, space, problemInstance::evaluate)

println("Steps taken: " + memory.stepCount)
println("Solution: " + finalSolution.solution.joinToString(", "))
println("Cost: " + finalSolution.cost)

In [ ]:
memory.stepCount